# JALA Data Scientist Take Home Test
- **Author:** Muhammad Arfian Praniza
- **Email:** fianpraniza@gmail.com
- **Linkedin:** https://www.linkedin.com/in/fianpraniza/
- **Contact:** +6281259676839

Import Libraries

In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Dict
import json
import os
import joblib
import pandas as pd
import numpy as np

In [2]:
model_dir = './models/'
sr_model = ('./models/sr_model.joblib')
abw_model = ('./models/abw_model.joblib')
scaler_sr = ('./models/sr_scaler.joblib')
scaler_abw = ('./models/abw_scaler.joblib')

In [3]:
def prepare_model_deployment():
    """
    Implementasi tahap 5: Model Deployment
    """
    print("\n" + "="*80)
    print("TAHAP 5: MODEL DEPLOYMENT")
    print("="*80)
    
    # 1. Definisi Predictor Class
    class ShrimpFarmPredictor:
        def __init__(self, sr_model, abw_model, sr_scaler, abw_scaler):
            self.sr_model = sr_model
            self.abw_model = abw_model
            self.sr_scaler = sr_scaler
            self.abw_scaler = abw_scaler
        
        def preprocess_features(self, data, target_type='sr'):
            scaler = self.sr_scaler if target_type == 'sr' else self.abw_scaler
            scaled_features = scaler.transform(data)
            return scaled_features
        
        def predict_sr(self, features):
            scaled_features = self.preprocess_features(features, 'sr')
            predictions = self.sr_model.predict(scaled_features)
            return np.clip(predictions, 0, 100)
        
        def predict_abw(self, features):
            scaled_features = self.preprocess_features(features, 'abw')
            predictions = self.abw_model.predict(scaled_features)
            return np.maximum(predictions, 0)
    
    # 2. Setup FastAPI
    app = FastAPI(title="Shrimp Farm Prediction API")
    
    class PredictionFeatures(BaseModel):
        features: Dict[str, float]
    
    # Inisialisasi predictor
    predictor = ShrimpFarmPredictor(
        sr_model=sr_model,
        abw_model=abw_model,
        sr_scaler=scaler_sr,
        abw_scaler=scaler_abw
    )
    
    @app.post("/predict/sr")
    async def predict_survival_rate(features: PredictionFeatures):
        try:
            prediction = predictor.predict_sr(pd.DataFrame([features.features]))
            return {"survival_rate": float(prediction[0])}
        except Exception as e:
            raise HTTPException(status_code=400, detail=str(e))
    
    @app.post("/predict/abw")
    async def predict_abw(features: PredictionFeatures):
        try:
            prediction = predictor.predict_abw(pd.DataFrame([features.features]))
            return {"average_body_weight": float(prediction[0])}
        except Exception as e:
            raise HTTPException(status_code=400, detail=str(e))
    
    return app

In [4]:
def save_deployment_files(model_dir):
    """
    Menyimpan file-file yang diperlukan untuk deployment
    """
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
        print(f"\nDibuat direktori baru: {model_dir}")
    
    # Save models
    joblib.dump(sr_model, os.path.join(model_dir, 'sr_model.joblib'))
    joblib.dump(abw_model, os.path.join(model_dir, 'abw_model.joblib'))
    joblib.dump(scaler_sr, os.path.join(model_dir, 'sr_scaler.joblib'))
    joblib.dump(scaler_abw, os.path.join(model_dir, 'abw_scaler.joblib'))
    
    # Save API specification
    api_spec = {
        "openapi": "3.0.0",
        "info": {
            "title": "Shrimp Farm Prediction API",
            "version": "1.0.0"
        },
        "paths": {
            "/predict/sr": {
                "post": {
                    "summary": "Prediksi Survival Rate",
                    "requestBody": {
                        "required": True,
                        "content": {
                            "application/json": {
                                "schema": {
                                    "type": "object",
                                    "properties": {
                                        "features": {
                                            "type": "object"
                                        }
                                    }
                                }
                            }
                        }
                    }
                }
            },
            "/predict/abw": {
                "post": {
                    "summary": "Prediksi Average Body Weight",
                    "requestBody": {
                        "required": True,
                        "content": {
                            "application/json": {
                                "schema": {
                                    "type": "object",
                                    "properties": {
                                        "features": {
                                            "type": "object"
                                        }
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    }
    
    with open(os.path.join(model_dir, 'api_specification.json'), 'w') as f:
        json.dump(api_spec, f, indent=2)
    
    # Save example usage
    example_usage = {
        "predict_sr": {
            "endpoint": "/predict/sr",
            "method": "POST",
            "example_request": {
                "features": {
                    "temperature": 30.5,
                    "do": 5.2,
                    "ph": 7.8,
                    "salinity": 20
                }
            }
        },
        "predict_abw": {
            "endpoint": "/predict/abw",
            "method": "POST",
            "example_request": {
                "features": {
                    "temperature": 30.5,
                    "do": 5.2,
                    "ph": 7.8,
                    "salinity": 20
                }
            }
        }
    }
    
    with open(os.path.join(model_dir, 'example_usage.json'), 'w') as f:
        json.dump(example_usage, f, indent=2)
    
    # Save requirements.txt
    requirements = [
        "fastapi==0.68.0",
        "uvicorn==0.15.0",
        "pandas==1.3.0",
        "numpy==1.21.0",
        "scikit-learn==0.24.2",
        "joblib==1.0.1"
    ]
    
    with open(os.path.join(model_dir, 'requirements.txt'), 'w') as f:
        f.write('\n'.join(requirements))
    
    print("\nFile deployment telah disimpan di:", model_dir)

In [5]:
# Jalankan deployment
try:
    print("\nMenyiapkan model deployment...")
    app = prepare_model_deployment()
    
    if 'model_dir' in locals():
        save_deployment_files(model_dir)
        print("\nProses deployment selesai!")
        
        print("\nUntuk menjalankan API, gunakan kode berikut:")
        print("uvicorn main:app --reload")
    else:
        print("\nError: model_dir tidak ditemukan")
except Exception as e:
    print(f"\nError dalam deployment: {str(e)}")


Menyiapkan model deployment...

TAHAP 5: MODEL DEPLOYMENT

File deployment telah disimpan di: ./models/

Proses deployment selesai!

Untuk menjalankan API, gunakan kode berikut:
uvicorn main:app --reload
